In [6]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="gemma2:latest",
    base_url= "100.108.14.46"
)

In [7]:
import json


docs = []

with open('FrenchParameters.json', 'r') as file:
    docs = json.load(file)

print(docs)

[{'parameterName': 'Nombre de Globules Blancs (WBC), leucocytes, globules blancs', 'description': 'Mesure le nombre total de globules blancs dans le sang, qui sont essentiels pour la réponse immunitaire et la lutte contre les infections.'}, {'parameterName': 'Nombre de Globules Rouges (RBC), érythrocytes, hématies', 'description': "Compte le nombre de globules rouges, responsables du transport de l'oxygène des poumons vers les tissus et de l'élimination du dioxyde de carbone."}, {'parameterName': 'Hémoglobine (Hb), pigment sanguin, oxygène sanguin', 'description': "Détermine la quantité d'hémoglobine dans le sang, une protéine des globules rouges qui se lie à l'oxygène pour le transporter dans tout le corps."}, {'parameterName': 'Hématocrite (Hct), volume cellulaire, érythrocrit', 'description': "Mesure la proportion de globules rouges dans le sang, indiquant la viscosité sanguine et la capacité de transport d'oxygène."}, {'parameterName': 'Volume Corpusculaire Moyen (MCV), taille moye

In [11]:
listOfParameters = ",".join(map(lambda x: x['parameterName'].split(',')[0], docs))
print(listOfParameters)

Nombre de Globules Blancs (WBC),Nombre de Globules Rouges (RBC),Hémoglobine (Hb),Hématocrite (Hct),Volume Corpusculaire Moyen (MCV),Teneur Moyenne en Hémoglobine Corpusculaire (MCH),Concentration Corpusculaire Moyenne en Hémoglobine (MCHC),Nombre de Plaquettes,Volume Plaquettaire Moyen (MPV),Largeur de Distribution des Globules Rouges (RDW),Neutrophiles,Lymphocytes,Monocytes,Éosinophiles,Basophiles,Sodium (Na),Potassium (K),Kaliémie (K),Chlorure (Cl),Bicarbonate (HCO3),Azote uréique sanguin (BUN),Créatinine,Glucose,Calcium (Ca),Albumine,Protéines totales,Phosphatase alcaline (ALP),Alanine Aminotransférase (ALT),Aspartate Aminotransférase (AST),Bilirubine,Cholestérol total,Lipoprotéine de basse densité (LDL),Lipoprotéine de haute densité (HDL),Triglycérides,Lipoprotéine de très basse densité (VLDL),Rapport Cholestérol/HDL,Alanine Aminotransférase (ALT),Aspartate Aminotransférase (AST),Phosphatase alcaline (ALP),Bilirubine (Total,Albumine,Protéines totales,Gamma-Glutamyl Transférase (GGT

In [8]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
import asyncio

url = "100.108.14.46"
client = QdrantClient(url)
collection_exists = client.collection_exists(
    collection_name="biologyParametersStore",
   
    )
if not collection_exists :
    client.create_collection(
        collection_name="biologyParametersStore",
        vectors_config=VectorParams(size=4096, distance=Distance.COSINE)
        )
    
vectors = []


    
for idx, doc in enumerate(docs):
    # Embed the 'content' property
    vector = await embeddings.aembed_query(doc["description"])
    
    # Add other properties as metadata in the payload
    payload = {
        "parameterName": doc['parameterName'],
        "description": doc['description']
    }
    vectors.append({'payload': payload, 'vector': vector, 'idx': idx})
    # Insert vector and metadata into Qdrant
print(vectors[0])

client.upsert(
    collection_name="biologyParametersStore",
    points=[
        PointStruct(id=vec['idx'], payload=vec['payload'], vector=vec['vector'])
        for vec in vectors
    ]
)



{'payload': {'parameterName': 'Nombre de Globules Blancs (WBC), leucocytes, globules blancs', 'description': 'Mesure le nombre total de globules blancs dans le sang, qui sont essentiels pour la réponse immunitaire et la lutte contre les infections.'}, 'vector': [-0.0070744622, -0.01202116, 0.00854713, -0.008155166, -0.00048813166, -0.002141251, 0.004299314, 0.017350178, 0.009505958, -0.00092705234, 0.00029038257, 0.003889447, 4.3140575e-05, -0.008866278, -0.023404941, -0.00055672263, -0.004129468, 0.0090084495, -0.004685348, -0.0005186252, 0.004531864, 0.008670068, 0.014364421, 0.0066446397, 0.00389317, 0.016367635, -0.018882738, 0.008738998, -0.0062228725, -0.011403775, -0.00092697015, -0.002228939, 0.009125073, 0.0038679785, -0.028438538, -0.012850019, 0.0035600015, 0.016768657, 0.008415155, -0.005244997, -0.019569091, -0.007207501, -0.016891094, -0.007710987, 0.02166505, -0.019278849, 0.011273706, -0.025413124, -0.0062864115, 0.0133247, 0.011711371, -0.0033074368, 0.004616732, -0.01

UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Wrong input: Vector dimension error: expected dim: 4096, got 3584"},"time":0.007589731}'

In [ ]:



query_vector = await embeddings.aembed_query('Kaliémie (K)')

results = client.search(
    collection_name="biologyParametersStore",
    query_vector=query_vector,
    limit=3,
    #score_threshold=0.5
)
formatted_results = [
    {
        "id": result.id,
        "score": result.score,
        "payload": result.payload,
    }
    for result in results
]
print(formatted_results)

[{'id': 142, 'score': 0.41125956, 'payload': {'parameterName': 'Niveau de C4, C4 complément', 'description': 'Le C4 est un autre composant du système du complément qui travaille avec le C3 pour promouvoir les réponses immunitaires. Des niveaux anormaux de C4 peuvent indiquer des troubles du système immunitaire tels que le lupus, ainsi que des infections aiguës et des inflammations.'}}, {'id': 141, 'score': 0.3860992, 'payload': {'parameterName': 'Niveau de C3, C3 complément', 'description': "Le C3 est un composant clé du système du complément, qui fait partie de la réponse immunitaire et aide à éliminer les agents pathogènes du corps. Ses niveaux peuvent indiquer l'activité du système immunitaire. Des niveaux faibles sont associés à des maladies auto-immunes, tandis que des niveaux élevés peuvent suggérer une infection ou une inflammation."}}, {'id': 11, 'score': 0.33247682, 'payload': {'parameterName': 'Lymphocytes, cellules immunitaires, globules blancs lymphocytaires', 'description'